# Machine Learning (Summer 2018)

## Practice Session 6

May, 22nd 2018

Ulf Krumnack

Institute of Cognitive Science
University of Osnabrück

## Today's Session

* exercise sheet 07
* principle component analysis (PCA)

# Principle Component Analysis (PCA)

Exercise 1 (ML-06 slides 7-12): What is the idea of dimension reduction. When does dimension reduction make sense? Discuss the topic with your neighbor. (5 min)

Exercise 2 (ML-06 slides 13-19): What is the idea of PCA? How does it work? Name two different ways to describe its objectives. (10 min)

Exercise 3 (ML-06 slides 20-22): What are eigenfaces? What are they used for? Execute the code in the following cells and try to understand what it is doing. (10 min)

# Eigenfaces

**a)** Import the images from the directory `images/trainimgs` into an numpy array using the function 
`read_images_from_directory` provided in the cell below. Display the images and the corresponding names.

In [ ]:
%matplotlib inline
import sys
import os
import glob
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt


def read_images_from_directory(directory, suffix, shape):
    """
    Read all images found in DIRECTORY with given file
    name SUFFIX. All images should have the same SHAPE,
    specified as (rows,columns).
    
    Returns:
        images: A numpy array of shape m*rows*columns (from shape)
        names: A list of corresponding image names.
    """

    # initialize the image array and name list
    #images = np.empty((0, *shape))
    images = np.empty((0, ) + shape)
    names = []

    # now loop through all image files in the directory
    for file_name in glob.glob(directory + os.sep + '*.' + suffix):
        if os.path.isfile(file_name):

            # load each image (as double)
            img = misc.imread(file_name, mode='F')

            # check for correct size
            if img.shape == shape:
                images = np.append(images, img.reshape((1, ) + shape), axis=0)
                names.append(os.path.basename(file_name))
            else:
                print(
                    'warning: Image "' + file_name +
                    '" with wrong size will be ignored!',
                    file=sys.stderr)

    return images, names


# image file suffix
suffix = 'pgm'

# image size
img_shape = (192, 168)

train_imgs, train_names = read_images_from_directory('trainimg','pgm', img_shape)

plt.figure(figsize=(12, 12))
plt.gray()
for i, n in enumerate(train_names):
    plt.subplot(5, 4, i + 1)
    plt.axis('off')
    plt.imshow(train_imgs[i])
    plt.title(n[5:7])
plt.show()

Principal components are the eigenvalues of the autocorrelation matrix of a dataset. The eigenvalues give the ordering of the components.

In [ ]:
import numpy as np

def pca(data):
    """
    Perform principal component analysis.
    
    Args:
        data (ndarray): an k*n dimensional array (k entries with n dimensions)
        
    Returns:
        ndarray: Array holding the principal components.
    """


    # Center the data (subtract the mean along columns).
    centered_data = data - data.mean(axis=0)

    covariance_matrix = np.cov(centered_data)
    
    # Computing eigenvalues and eigenvectors of covariance matrix.
    # Attention: not always sorted.
    eigenvals, eigenvecs = np.linalg.eig(covariance_matrix)
    # projection of the data in the new space
    pc = eigenvecs.T @ centered_data  

    return pc

**b)** Use PCA to compute the eigenfaces (i.e. the eigenvectors of the face images). Explain what kind of input PCA expects, and how that fits to our images (you may have to `reshape` the images!). Finally, display the eigenfaces.

In [ ]:
face_vecs = train_imgs.reshape((-1, np.prod(img_shape)))
eigenfaces = pca(face_vecs)

plt.figure(figsize=(12, 12))
plt.gray()
for i, eigenface in enumerate(eigenfaces):
    plt.subplot(5, 4, i + 1)
    plt.axis('off')
    plt.imshow(eigenface.reshape(img_shape))
    plt.title('Eigenface {}'.format(i))
plt.show()

**c)** Now project the training face images into the eigenspace to calculate their ”feature vectors”,
i.e. a representation with significantly lower dimension. For the projection of the face images,
they have to be centered first, i.e. the mean face vector has to be subtracted. Store the mean face in some vector (`mean_face`) and the representation achieved in some array (`face_db`). Finally restore the images from `face_db` and display them alongside the original image. Try out the effect of changing the number of eigenfaces to be used (`num_eigenfaces`).

In [ ]:
# number of eigenfaces to be used
num_eigenfaces = 10

# Compute the mean face.
mean_face = face_vecs.mean(axis=0)

# Reduce the number of eigenfaces to achieve more dimensionality reduction.
eigenfaces_used = eigenfaces[:num_eigenfaces]

# 3. Project images into the eigenface space and store
#    them in a "eigenface database":
face_db = (face_vecs - mean_face) @ eigenfaces_used.T

# ... and display the original image and the stored image
plt.figure(figsize=(12, 12))
plt.gray()
for i, face in enumerate(face_db):
    restored = (face[np.newaxis] @ eigenfaces_used) + mean_face
    plt.subplot(5, 8, 2 * i + 1)
    plt.axis('off')
    plt.imshow(train_imgs[i])
    plt.title('original')
    plt.subplot(5, 8, 2 * i + 2)
    plt.axis('off')
    plt.imshow(restored.reshape(img_shape))
    plt.title('restored')
plt.show()

**d)** Implement the function `recognize_face` that recognizes a face from that database by calculating the euclidean distance of this face feature vector to all of the training feature vectors from the database. The feature vector with the smallest distance represents the winner category. Check your implementation by recognizing the images from the training set (they should be recognized without error).

In [ ]:
from scipy.spatial.distance import cdist

def recognize_face(face, eigenfaces, mean_face, face_db):
    """
    Recognize a face from a face database.
    and return the index of the best matching database entry.

    The FACE is first centered and projected into the eigeface
    space provided by EIGENFACES. Then the best match is found
    according to the euclidean distance in the eigenface space.
    """
    index = -1

    # center the face
    centered = face - mean_face

    # and project it into the eigenface space
    projected = eigenfaces @ centered

    # Now compute the similarity to all known faces
    # (comparison is performed in the eigenface space)
    distances = cdist(face_db, projected[None, :])
    index = distances.argmin()

    return index


# ... and now check your function on the training set ...
def show_recognition_results(imgs, labels, train_imgs, train_labels,
                             num_eigenfaces, eigenfaces, mean_face, face_db):
    img_shape = imgs[0].shape
    plt.figure(figsize=(12, 12))
    plt.suptitle(
        'Face recognition based on {} principal components'.format(num_eigenfaces))
    plt.gray()
    for j, img in enumerate(imgs):

        # find the best match in the eigenface database
        winner = recognize_face(
            img.reshape(np.prod(img_shape)), eigenfaces, mean_face, face_db)
        name_label = labels[j][5:7]
        name_winner = train_labels[winner][5:7]

        plt.subplot(5, 8, 2 * j + 1)
        plt.axis('off')
        plt.imshow(img)
        plt.title(labels[j][5:7])

        plt.subplot(5, 8, 2 * j + 2)
        plt.axis('off')
        plt.imshow(train_imgs[winner])
        plt.title(('*' if name_label != name_winner else '') + name_winner)
    plt.show()
    
show_recognition_results(train_imgs, train_names,
                         train_imgs, train_names,
                         num_eigenfaces, eigenfaces_used, 
                         mean_face, face_db)

**e)** Now classify the images in directory `testimg/`. Try to reduce the number of principal components
used. How many PCs are necessary to still achieve perfect classification?

In [ ]:
test_imgs, test_names = read_images_from_directory('testimg', suffix, img_shape)

show_recognition_results(test_imgs, test_names,
                         train_imgs, train_names,
                         num_eigenfaces, eigenfaces_used, 
                         mean_face, face_db)

# Sheet 05, Assignment 2: Implement and Apply PCA


## Example: the `cars` dataset

* simplified from the JSE [2004 New Car and Truck Data](http://www.amstat.org/publications/jse/jse_data_archive.htm))
* measurements taken on 97 different cars
* eleven features: Suggested retail price (USD), Price to dealer (USD), Engine size (liters), Number of engine cylinders, Engine horsepower, City gas mileage, Highway gas mileage, Weight (pounds), Wheelbase (inches), Length (inches) and Width (inches). 

Goal: visualize these high dimensional features to get a feeling for how the cars relate to each other so we need to find a subspace of dimension two or three into which we can project the data.

In [ ]:
import numpy as np

# Load the cars dataset in cars.csv .
cars = np.loadtxt('cars.csv', delimiter=',')

assert cars.shape == (97, 11), "Shape is not (97, 11), was {}".format(cars.shape)

Try to get some initial idea of the dataset:

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()
cols = ['Suggested retail price (USD)', 'Price to dealer (USD)', 
          'Engine size (liters)', 'Number of engine cylinders', 
          'Engine horsepower', 'City gas mileage' , 
          'Highway gas mileage', 'Weight (pounds)', 
          'Wheelbase (inches)', 'Length (inches)', 'Width (inches)']

df = pd.DataFrame(cars, columns=cols)
sns.pairplot(df)

First step: normalize the data (such that they have a zero mean and a unit standard deviation)

$$\frac{X - \mu}{\sigma}$$

* $X$ is an $m \times n$ matrix with $n=11$ features and $m=97$ samples
* $\mu$ is the mean of that dataset, a $n$-dimensional vector
* $\sigma$ is the standard deviation, again in each of the $n$ feature dimensions

In [ ]:
import numpy as np

# Normalize the data and store them in a variable called cars_norm.
cars_norm = (cars - np.mean(cars, axis=0)) / np.std(cars, axis=0)

# Alternatively one could use:
# import sklearn.preprocessing
# cars_norm = sklearn.preprocessing.scale(cars)

assert cars_norm.shape == (97, 11), "Shape is not (97, 11), was {}".format(cars.shape)
assert np.abs(np.sum(cars_norm)) < 1e-10, "Absolute sum was {} but should be close to 0".format(np.abs(np.sum(cars_norm)))
assert np.abs(np.sum(cars_norm ** 2) / cars_norm.size - 1) < 1e-10, "The data is not normalized, sum/N was {} not 1".format(np.sum(cars_norm ** 2) / cars_norm.size)

Find a subspace that maximizes the variance:
* determine the eigenvectors of the covariance matrix
* for normalized data the covariance matrix is calculated as $C = X^T\cdot X$
* the entry $c_{i,j}$ in $C$ tells how much feature $i$ correlates with feature $j$

Note: sometimes the formula $C=X\cdot X^T$ can be found: depends on how the data is stored!

In [ ]:
import numpy as np

# Compute the autocovariance matrix and store it into autocovar
autocovar = cars_norm.T @ cars_norm

assert autocovar.shape == (11, 11)

# Compute the eigenvalues und eigenvectors and store them into eigenval and eigenvec
# (Figure out a function to do this for you)
eigenval, eigenvec = np.linalg.eigh(autocovar)

assert eigenval.shape == (11,)
assert eigenvec.shape == (11, 11)

Plot the spectrum of the eigenvalues
* make sure that they are sorted by their magnitude!
* how many principal components should we include based on the spectrum plot?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Sort eigenvectors (and -values) by descending order of eigenvalues.
sort = np.argsort(-eigenval)
eigenval = eigenval[sort]
eigenvec = eigenvec[:,sort]

# To get an idea of the eigenvalues we plot them.
figure = plt.figure('Eigenvalue comparison')
plt.bar(np.arange(len(eigenval)), eigenval)
plt.show()

## Application: Visualization

Poject data into a low dimensional subspace (e.g., a 2D space)

In [ ]:
# Project the data down into the two dimensional subspace
#proj = cars_norm @ eigenvec[:,0:2]
proj = cars_norm @ eigenvec

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot projected data
plt.figure('Data projected onto first two Principal Components')
plt.xlim(-8, 8)
plt.ylim(-4, 7)
plt.scatter(proj[:,0], proj[:,1])
# Divide plot into quadrants
plt.axhline(0, color='green')
plt.axvline(0, color='green')
plt.show()

#### PCA is good
* only few points overlap and the points are generally spread out well in the subspace
* there is not much trend in the plot which is what we desired, i.e. the axes are not redundant
* no clusters can be recognized. 

* what kinds of cars are in the four quadrants of the first plot?
* was it justifiable that we only considered the first two principle components?

plot the two first principle component vectors as eleven two dimensional points
* how are the features are projected into the subspace?

In [ ]:
# Plot eigenvectors
plt.figure('Eigenvector plot',figsize=(12,8))
plt.scatter(eigenvec[:,0], eigenvec[:,1])
plt.axhline(0, color='green')
plt.axvline(0, color='green')

# add labels
labels = ['Suggested retail price (USD)', 'Price to dealer (USD)', 
          'Engine size (liters)', 'Number of engine cylinders', 
          'Engine horsepower', 'City gas mileage' , 
          'Highway gas mileage', 'Weight (pounds)', 
          'Wheelbase (inches)', 'Length (inches)', 'Width (inches)']
for label, x, y in zip(labels, eigenvec[:,0], eigenvec[:,1]):
    plt.annotate(
        label, xy = (x, y), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'left', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'blue', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
plt.show()

# Sheet 05, Assignment 3: PCA

PCA finds the subspace that captures most of the data variance:
* the orthonormal projection onto an $m$-dimensional subspace that maximizes the variance of the projected data is defined by the principal components
* two steps:
  1. a one-dimensional subspace
  1. an $m$-dimensional space

## Step 1: a one-dimensional subspace

Goal:
* determine a (unit) vector $\vec{p}$, such that the variance of the data, when projected onto the subspace determined by that vector, is maximal.

* variance of the projected data: $\vec{p}^{T}C\vec{p}$ (with $C$ = autocorrelation matrix)
* maximize this expression
* avoid $\|\vec{p}\|\to\infty$: only consider unit vectors, i.e. we constrain $\vec{p}$ to be normalized: $\vec{p}^T\vec{p}=1$. 

Maximize the expression with this constraint (which can be done using a Lagrangian multiplier).

We want to maximize the expression
$$\vec{p}^T C\vec{p} + \lambda(1-\vec{p}^T\vec{p})$$
with respect to $\vec{p}$,

We have to find solutions for
$$\frac{\partial}{\partial\vec{p}}\left[ \vec{p}^T C\vec{p} + \lambda(1-\vec{p}^T\vec{p})\right] = 0$$

This leads to the equation
$$2C\cdot\vec{p} = 2\lambda\cdot\vec{p}$$
and hence
$$C\cdot\vec{p} = \lambda\cdot\vec{p}$$

in other words: for a vector $\vec{p}$ to maximize our expression, it has to be an eigenvector $C$ and $\lambda$ has to be the corresponding eigenvalue.

By left multiplying with $\vec{p}^T$ and using the fact that $\vec{p}^T\vec{p}=1$, we gain
$$\vec{p}^TC\vec{p}=\lambda$$
i.e. the projected variance will correspond to the eigenvalue $\lambda$ and hence is maximized when choosing the largest eigenvalue.

## Step 2: an $m$-dimensional projection space

Idea: use an inductive argument:
* assume the statement has been shown for the $(m-1)$-dimensional projection space, spanned by the $m-1$ (orthonormal) eigenvectors $\vec{p}_1,\ldots,\vec{p}_{m-1}$ corresponding to the $(m-1)$ largest eigenvalues $\lambda_1,\ldots,\lambda_{m-1}$.
* then find a (unit) vector $\vec{p}_m$, orthogonal to the existing vectors $\vec{p}_1,\ldots,\vec{p}_{m-1}$, that maximizes the projected variance $\vec{p}_m^TC\vec{p}_m$. 

First, the vector $\vec{p}_m$ should be linearly independent from $\vec{p}_1,\ldots,\vec{p}_{m-1}$, as it should define the new $m$-th dimension.

This property can be enforced by the (stronger) requirement that $\vec{p}_{m}$ should be orthogonal to
$\vec{p}_1,\ldots,\vec{p}_{m-1}$, i.e. 

$$\vec{p}_m^T\vec{p}_{i}=0 \text{ for } i=1,\ldots,m-1,$$

Espress this constraints by additionalLagrange multipliers $\eta_1,\ldots,\eta_{m-1}$.

Again, the variance in direction $\vec{p}_m$ is given by
$$\vec{p}_{m}^TC\vec{p}_{m}.$$

We want to maximize this value, again with the additional constraint that $\vec{p}_{m}$ is normalized, i.e.

$$\vec{p}_{m}^T\vec{p}_m=1,$$

which will be expressed by an additional Lagrange multiplier $\lambda_M$.

So in total we want to maximize the function
$$\vec{p}_{m}^TC\vec{p}_{m} + \sum_{i=1}^{m-1}\eta_i\vec{p}_m^T\vec{p}_{i} + \lambda_m(1-\vec{p}_{m}^T\vec{p}_{m})$$
with respect to $\vec{p}_m$,

i.e. we have to find solutions for
\begin{align}
  0
  & = \frac{\partial}{\partial\vec{p}_m}\left[\vec{p}_{m}^TC\vec{p}_{m} 
  + \sum_{i=1}^{m-1}\eta_i\vec{p}_m^T\vec{p}_{i}
  + \lambda_m(1-\vec{p}_{m}^T\vec{p}_m)\right] \\
  & = 2C\vec{p}_m + \sum_{i=1}^{m-1}\eta_i\vec{p}_{i} - 2\lambda_m\vec{p}_{m}
\end{align}

Multiplying this equation with $\vec{p}_{j}^T$ from the left yields (due to the orthogonality constraint)
\begin{align}
  0 = \vec{p}_{j}^T 0
  & = \vec{p}_{j}^T 2C\vec{p}_m +
  \vec{p}_{j}^T \sum_{i=1}^{m-1}\eta_i\vec{p}_{i} -
  \vec{p}_{j}^T 2\lambda_m\vec{p}_{m} \\
  &= 0 + \eta_j\vec{p}_{j}^T \vec{p}_{j}- 0 \\
  & = \eta_j
\end{align}
for $j=1,\ldots,m-1$.

So the problem simplifies to
$$0 = 2C\vec{p}_m - 2\lambda_m\vec{p}_{m}$$
i.e.,
$$C\vec{p}_m = \lambda_m\vec{p}_{m}$$

which just means that $\vec{p}_m$ has to be an eigenvector of the matrix $C$ with eigenvalue $\lambda_M$. 

Again, there may be multiple eigenvectors for $C$, so we have to select $\vec{p}_m$ in a way that it maximizes the variance in direction $\vec{p}_m$, i.e. the value

$$\vec{p}_{m}^TC\vec{p}_{m} = \vec{p}_{m}^T\lambda_M\vec{p}_{m} = \lambda_M.$$

This just means that we have to choose $\vec{p}_m$ to be the eigenvector with the largest eigenvalue (amongst those not previously selected). This completes the inductive step.